In [1]:
import csv
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2022-06-21 12:51:32.877793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 12:51:32.877851: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [15]:
with open("./dataset/BBC-News-Train.csv", 'r') as csvfile:
    print(f"First line (header) looks like this:\n\n{csvfile.readline()}")
    print(f"Each data point looks like this:\n\n{csvfile.readline()}")

First line (header) looks like this:

ArticleId,Text,Category

Each data point looks like this:

1833,worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002.

In [74]:
# GRADED FUNCTION: remove_stopwords
def remove_stopwords(sentence):
    # List of stopwords
    stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    
    # Sentence converted to lowercase-only
    sentence = sentence.lower()
    
    ### START CODE HERE
    sentence = sentence.split(" ")
    results = []
    for word in sentence:
        if word not in stopwords:
            results.append(word)
    
    sentence = ' '.join(results)
    
    ### END CODE HERE
    return sentence

In [75]:
sentence = '1833, worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebber s lawyers have said he was unaware of the fraud  arguing that auditors did not alert him to any problems.  ms cooper also said that during shareholder meetings mr ebbers often passed over technical questions to the company s finance chief  giving only  brief  answers himself. the prosecution s star witness  former worldcom financial chief scott sullivan  has said that mr ebbers ordered accounting adjustments at the firm  telling him to  hit our books . however  ms cooper said mr sullivan had not mentioned  anything uncomfortable  about worldcom s accounting during a 2001 audit committee meeting. mr ebbers could face a jail sentence of 85 years if convicted of all the charges he is facing. worldcom emerged from bankruptcy protection in 2004  and is now known as mci. last week  mci agreed to a buyout by verizon communications in a deal valued at $6.75bn.,busines'

In [76]:
remove_stopwords(sentence)

'1833, worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers battery fraud charges called company whistleblower first witness.  cynthia cooper  worldcom s ex-head internal accounting  alerted directors irregular accounting practices us telecoms giant 2002. warnings led collapse firm following discovery $11bn (£5.7bn) accounting fraud. mr ebbers pleaded not guilty charges fraud conspiracy.  prosecution lawyers argued mr ebbers orchestrated series accounting tricks worldcom  ordering employees hide expenses inflate revenues meet wall street earnings estimates. ms cooper  now runs consulting business  told jury new york wednesday external auditors arthur andersen approved worldcom s accounting early 2001 2002. said andersen given  green light  procedures practices used worldcom. mr ebber s lawyers said unaware fraud  arguing auditors not alert problems.  ms cooper also said shareholder meetings mr ebbers often passed technical questions company s finance 

In [77]:
def parse_data_from_file(filename):
    
    sentences = []
    labels = []
    with open(filename, 'r') as csvfile:
        ### START CODE HERE
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        
        for row in reader:
            labels.append(row[0])
            sentence = remove_stopwords(row[1])
            sentences.append(sentence)
            
        ### END CODE HERE
    return sentences, labels

In [79]:
# Test your function
sentences, labels = parse_data_from_file("./dataset/BBC-News-Train.csv")

print(f"There are {len(sentences)} sentences in the dataset.\n")
print(f"First sentence has {len(sentences[0].split())} words (after removing stopwords).\n")
print(f"There are {len(labels)} labels in the dataset.\n")
print(f"The first 5 labels are {labels[:5]}")

There are 1490 sentences in the dataset.

First sentence has 203 words (after removing stopwords).

There are 1490 labels in the dataset.

The first 5 labels are ['1833', '154', '1101', '1976', '917']


In [80]:
def fit_tokenizer(sentences):
    """
    Instantiates the Tokenizer class
    
    Args:
        sentences (list): lower-cased sentences without stopwords
    
    Returns:
        tokenizer (object): an instance of the Tokenizer class containing the word-index dictionary
    """
    ### START CODE HERE
    # Instantiate the Tokenizer class by passing in the oov_token argument
    tokenizer = Tokenizer(oov_token="<OOV>")
    # Fit on the sentences
    # Generate the word index dictionary
    tokenizer.fit_on_texts(sentences)
    
    ### END CODE HERE
    return tokenizer

In [81]:
tokenizer = fit_tokenizer(sentences)
word_index = tokenizer.word_index

print(f"Vocabulary contains {len(word_index)} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")

Vocabulary contains 24963 words

<OOV> token included in vocabulary


In [82]:
def get_padded_sequences(tokenizer, sentences):
    
    ### START CODE HERE
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    
    # Pad the sequences using the post padding strategy
    padded_sequences = pad_sequences(sequences, padding='post')
    ### END CODE HERE
    
    return padded_sequences

In [83]:
padded_sequences = get_padded_sequences(tokenizer, sentences)
print(f"First padded sequence looks like this: \n\n{padded_sequences[0]}\n")
print(f"Numpy array of all sequences has shape: {padded_sequences.shape}\n")
print(f"This means there are {padded_sequences.shape[0]} sequences in total and each one has a size of {padded_sequences.shape[1]}")

First padded sequence looks like this: 

[1322 1180  592 ...    0    0    0]

Numpy array of all sequences has shape: (1490, 1881)

This means there are 1490 sequences in total and each one has a size of 1881


In [84]:
padded_sequences

array([[ 1322,  1180,   592, ...,     0,     0,     0],
       [  773,   158,   887, ...,     0,     0,     0],
       [   45,   727, 10061, ...,     0,     0,     0],
       ...,
       [ 1795,   322,  1483, ...,     0,     0,     0],
       [  516,  1347,   597, ...,     0,     0,     0],
       [ 8683,  2912,   673, ...,     0,     0,     0]], dtype=int32)

In [85]:
def tokenize_labels(labels):
    ### START CODE HERE
    
    # Instantiate the Tokenizer class
    # No need to pass additional arguments since you will be tokenizing the labels
    label_tokenizer = Tokenizer()
    
    # Fit the tokenizer to the labels
    label_tokenizer.fit_on_texts(labels)
    
    # Save the word index
    label_word_index = label_tokenizer.word_index
    
    # Save the sequences
    label_sequences = label_tokenizer.texts_to_sequences(labels)

    ### END CODE HERE
    
    return label_sequences, label_word_index

In [86]:
label_sequences, label_word_index = tokenize_labels(labels)
print(f"Vocabulary of labels looks like this {label_word_index}\n")
print(f"First ten sequences {label_sequences[:10]}\n")

Vocabulary of labels looks like this {'1833': 1, '154': 2, '1101': 3, '1976': 4, '917': 5, '1582': 6, '651': 7, '1797': 8, '2034': 9, '1866': 10, '1683': 11, '1153': 12, '1028': 13, '812': 14, '707': 15, '1588': 16, '342': 17, '486': 18, '1344': 19, '1552': 20, '1547': 21, '177': 22, '1785': 23, '1617': 24, '405': 25, '1561': 26, '702': 27, '1026': 28, '1527': 29, '1503': 30, '1951': 31, '1407': 32, '2002': 33, '2100': 34, '466': 35, '687': 36, '1009': 37, '805': 38, '771': 39, '1532': 40, '2205': 41, '2000': 42, '953': 43, '1394': 44, '1522': 45, '455': 46, '593': 47, '590': 48, '277': 49, '90': 50, '904': 51, '527': 52, '1763': 53, '42': 54, '1364': 55, '1418': 56, '643': 57, '40': 58, '1518': 59, '2046': 60, '464': 61, '180': 62, '476': 63, '2017': 64, '315': 65, '96': 66, '1079': 67, '947': 68, '1742': 69, '972': 70, '210': 71, '2144': 72, '1766': 73, '1971': 74, '1303': 75, '1638': 76, '79': 77, '1055': 78, '1804': 79, '1929': 80, '371': 81, '445': 82, '105': 83, '1297': 84, '1932